# Consolidated Data Exporter to Excel

This notebook gathers key data tables generated by the credit risk analysis system and exports them into a single Excel file (`consolidated_data_export.xlsx`) with multiple sheets. It also displays previews of these tables within the notebook itself.

In [ ]:
import pandas as pd
from pathlib import Path
import json # For pretty printing dicts if needed, not strictly for excel
import numpy as np # For synthetic data generation
import logging

# Project-specific imports (adjust paths if notebook is run from a different CWD)
# Assuming notebook is in 'notebooks/' and project root is parent.
import sys
if '../' not in sys.path:
    sys.path.append('../') # Add project root to path

from src.data_management.knowledge_base import KnowledgeBaseService, IndustrySector, Currency, CollateralType
from src.data_management.ontology import CorporateEntity # For type hinting if needed
from src.risk_models.pd_model import PDModel
from src.risk_models.lgd_model import LGDModel
from src.mlops.model_registry import ModelRegistry
from src.data_management.knowledge_graph import KnowledgeGraphService # kg_service needed for RiskMapService
from src.risk_map.risk_map_service import RiskMapService

# Configure basic logging for the notebook
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger('ExcelExporterNotebook')

# Ensure output directory exists
output_dir = Path("../output")
output_dir.mkdir(exist_ok=True)

logger.info("Setup complete. Modules imported.")

In [ ]:
logger.info("Initializing services and loading models...")
kb_service = KnowledgeBaseService()

# Load PD Model (with fallback to training for demo robustness)
registry = ModelRegistry()
pd_model_path_str = registry.get_production_model_path("PDModel")
pd_model_instance = PDModel(model_path=Path(pd_model_path_str) if pd_model_path_str else None)
if not pd_model_instance.load_model():
    logger.warning("PD Model could not be loaded from registry or default path. Training a new one for demo...")
    if kb_service.get_all_loans(limit=1): # Check if data is available
        train_metrics_pd = pd_model_instance.train(kb_service) # This will also register it
        if "error" not in train_metrics_pd:
            logger.info(f"Newly trained PD Model. Metrics: {train_metrics_pd}")
            latest_pd_versions = registry.list_models("PDModel")
            if latest_pd_versions:
                registry.update_model_status("PDModel", latest_pd_versions[0]['model_version'], "production")
                logger.info(f"Newly trained PD Model {latest_pd_versions[0]['model_version']} set to production for this session.")
        else:
            logger.error(f"Failed to train PD Model for demo: {train_metrics_pd.get('error', 'Unknown error')}")
    else:
        logger.error("ERROR: KB data not loaded, cannot train PD Model for demo.")
else:
    logger.info(f"PD Model loaded successfully from: {pd_model_instance.model_path}")

# Load LGD Model (with fallback to training)
lgd_model_path_str = registry.get_production_model_path("LGDModel")
lgd_model_instance = LGDModel(model_path=Path(lgd_model_path_str) if lgd_model_path_str else None)
if not lgd_model_instance.load_model():
    logger.warning("LGD Model could not be loaded from registry or default path. Training a new one for demo...")
    if kb_service.get_all_loans(limit=1): # Check if data is available
        train_metrics_lgd = lgd_model_instance.train(kb_service) # This will also register it
        if "error" not in train_metrics_lgd:
            logger.info(f"Newly trained LGD Model. Metrics: {train_metrics_lgd}")
            latest_lgd_versions = registry.list_models("LGDModel")
            if latest_lgd_versions:
                registry.update_model_status("LGDModel", latest_lgd_versions[0]['model_version'], "production")
                logger.info(f"Newly trained LGD Model {latest_lgd_versions[0]['model_version']} set to production for this session.")
        else:
            logger.error(f"Failed to train LGD Model for demo: {train_metrics_lgd.get('error', 'Unknown error')}")
    else:
        logger.error("ERROR: KB data not loaded, cannot train LGD Model for demo.")
else:
    logger.info(f"LGD Model loaded successfully from: {lgd_model_instance.model_path}")

kg_service = KnowledgeGraphService(kb_service=kb_service)
risk_map_service = RiskMapService(kb_service=kb_service, pd_model=pd_model_instance, lgd_model=lgd_model_instance, kg_service=kg_service)

logger.info("Services and models initialized.")

### 1. Portfolio Risk Overview

In [ ]:
logger.info("Generating Portfolio Risk Overview...")
portfolio_overview_data = risk_map_service.generate_portfolio_risk_overview()
portfolio_overview_df = pd.DataFrame(portfolio_overview_data)
logger.info(f"Portfolio Overview Table Shape: {portfolio_overview_df.shape}")
if not portfolio_overview_df.empty:
    display(portfolio_overview_df.head())
else:
    logger.warning("Portfolio Overview DataFrame is empty.")

### 2. Risk Summary by Sector

In [ ]:
logger.info("Generating Risk Summary by Sector...")
sector_summary_data = risk_map_service.get_risk_summary_by_sector(portfolio_overview_data)
sector_summary_df = pd.DataFrame.from_dict(sector_summary_data, orient='index')
logger.info(f"Sector Summary Table Shape: {sector_summary_df.shape}")
if not sector_summary_df.empty:
    display(sector_summary_df.head())
else:
    logger.warning("Sector Summary DataFrame is empty.")

### 3. Risk Summary by Country

In [ ]:
logger.info("Generating Risk Summary by Country...")
country_summary_data = risk_map_service.get_risk_summary_by_country(portfolio_overview_data)
country_summary_df = pd.DataFrame.from_dict(country_summary_data, orient='index')
logger.info(f"Country Summary Table Shape: {country_summary_df.shape}")
if not country_summary_df.empty:
    display(country_summary_df.head())
else:
    logger.warning("Country Summary DataFrame is empty.")

### 4. Sample Company Data (from Knowledge Base)

In [ ]:
logger.info("Fetching company data from Knowledge Base...")
all_companies_full_obj = kb_service.get_all_companies()
companies_data_for_excel = [comp.model_dump(mode='json') for comp in all_companies_full_obj]
companies_df = pd.DataFrame(companies_data_for_excel)

# Convert list-like fields to comma-separated strings for better Excel view
list_cols_to_convert = ['subsidiaries', 'suppliers', 'customers', 'loan_agreement_ids', 'financial_statement_ids']
for col in list_cols_to_convert:
    if col in companies_df.columns:
        companies_df[col] = companies_df[col].apply(lambda x: ', '.join(x) if isinstance(x, list) and x else None)

logger.info(f"Companies Table Shape: {companies_df.shape}")
if not companies_df.empty:
    display(companies_df.head())
else:
    logger.warning("Companies DataFrame from KB is empty.")

### 5. Illustrative Synthetic Equities Data

In [ ]:
def generate_synthetic_equities(num_entities=10):
    data = []
    sectors = [s.value for s in IndustrySector] + ["Consumer Discretionary", "Healthcare", "Real Estate"]
    for i in range(num_entities):
        beta = np.random.normal(1.0, 0.3)
        data.append({
            'entity_id': f"EQ_COMP{i:03d}",
            'name': f"Equity Example Co {i+1}",
            'sector': np.random.choice(sectors),
            'simulated_market_cap': np.random.lognormal(mean=np.log(10000), sigma=1.5) * 1_000_000, # In Millions
            'simulated_beta': round(beta, 2),
            'simulated_expected_return': round(0.02 + beta * 0.05, 4), # risk_free + beta * market_premium
            'simulated_volatility': round(np.random.uniform(0.15, 0.60), 4)
        })
    return pd.DataFrame(data)

logger.info("Generating Illustrative Synthetic Equities Data...")
synthetic_equities_df = generate_synthetic_equities()
logger.info(f"Synthetic Equities Table Shape: {synthetic_equities_df.shape}")
display(synthetic_equities_df.head())

### 6. Illustrative Synthetic Commodities Data

In [ ]:
def generate_synthetic_commodities(num_assets=4):
    commodities = ["Crude Oil", "Gold", "Copper", "Corn"]
    data = []
    weights = np.random.dirichlet(np.ones(num_assets), size=1)[0]
    for i, name in enumerate(commodities[:num_assets]):
        data.append({
            'asset_id': f"COMM_{name.replace(' ', '_').upper()[:4]}",
            'name': name,
            'simulated_expected_return': round(np.random.uniform(0.03, 0.08), 4),
            'simulated_volatility': round(np.random.uniform(0.10, 0.40), 4),
            'simulated_weight_in_portfolio': round(weights[i], 4)
        })
    return pd.DataFrame(data)

logger.info("Generating Illustrative Synthetic Commodities Data...")
synthetic_commodities_df = generate_synthetic_commodities()
logger.info(f"Synthetic Commodities Table Shape: {synthetic_commodities_df.shape}")
display(synthetic_commodities_df.head())

### Exporting Data to Excel

In [ ]:
excel_file_path = output_dir / "consolidated_data_export.xlsx"
logger.info(f"Attempting to export data to: {excel_file_path}")
try:
    with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
        if not portfolio_overview_df.empty:
            portfolio_overview_df.to_excel(writer, sheet_name='PortfolioOverview', index=False)
        if not sector_summary_df.empty:
            sector_summary_df.to_excel(writer, sheet_name='SectorSummary', index=True)
        if not country_summary_df.empty:
            country_summary_df.to_excel(writer, sheet_name='CountrySummary', index=True)
        if not companies_df.empty:
            companies_df.to_excel(writer, sheet_name='CompaniesKB', index=False)
        if not synthetic_equities_df.empty:
            synthetic_equities_df.to_excel(writer, sheet_name='SyntheticEquities', index=False)
        if not synthetic_commodities_df.empty:
            synthetic_commodities_df.to_excel(writer, sheet_name='SyntheticCommodities', index=False)
    logger.info(f"Data successfully exported to {excel_file_path}")
except Exception as e:
    logger.error(f"Error exporting to Excel: {e}")
    logger.error("Please ensure 'openpyxl' is installed: pip install openpyxl")

This notebook has generated the `consolidated_data_export.xlsx` file in the `output/` directory. It contains key tables from the credit risk analysis system. The previews above show the structure of each sheet.